In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from numpy.linalg import matrix_power as MP

import random
import scipy
from scipy.spatial.distance import cdist
from scipy.io import savemat, loadmat

import networkx as nx
import sys

import tensorflow as tf
from tensorflow.math import scalar_mul as c_mul
from keras.callbacks import EarlyStopping

print(f"TF Version: {tf.__version__}")

TF Version: 2.14.0


In [2]:
ADHD_data = [np.array(d).T for d in loadmat("./Dataset/ADHD_Data.mat")['ADHD_Data'][0, :]]
ADHD_label = [0] * 20 + [1] * 20 # 0 - Control, 1 - ADHD

X_train_data = ADHD_data
y_train_data = ADHD_label

num_samples = len(X_train_data)
N = X_train_data[0].shape[0]
T_list = [X_train_data[i].shape[1] for i in range(num_samples)]
T = T_list[0]
print(f"Number of Samples: {num_samples}, N: {N}")

Number of Samples: 40, N: 39


In [3]:
X_train_data = [(data - np.mean(data.flatten())) / np.std(data.flatten()) for data in X_train_data]

In [4]:
def generate_delta(T):
    diff_matrix = np.zeros([T, T - 1]) - np.eye(T, T - 1)
    for i in range(1, T):
        diff_matrix[i, i - 1] = 1
    return tf.convert_to_tensor(diff_matrix, dtype = "float32")

def adj_to_laplacian(AdjL):
    tmp = np.array(AdjL, dtype = "float32")
    return np.diag(np.sum(tmp, axis = 0)) - tmp

def initial_graph(X_training, N, top_values = 6):
    distance_matrix = cdist(X_training, X_training, metric = 'euclidean')
    
    # top_values = 6
    sorted_indices = distance_matrix.argsort(1)
    TAdj = np.zeros([N, N])
    
    for i in range(N):
        for j in range(1, top_values):
            k = sorted_indices[i][j]
            TAdj[i][k] = 1
    
    TAdj = (TAdj + TAdj.T) / 2
    TAdj = np.where(TAdj > 0.1, 1, 0)
    
    Laplacian_init = np.diag(np.sum(TAdj, 1)) - TAdj
    
    return Laplacian_init

def label_print(idx, length):
    l = len(str(idx))
    if l < length:
        return "0" * (length - l) + str(idx)
    else:
        return str(idx)

In [5]:
Mul = np.eye(N) - np.ones([N, N])
Mul = tf.convert_to_tensor(Mul, dtype = "float32")

In [6]:
coeffs = 0
psi_cost = 0
Laplacian = 0
A_transform_cost = 0

model_pred = []
loss_model = []
learned_graph = []
mse_known_list = []
mse_unknown_list = []

In [7]:
coeffs_list = np.load("./Outputs/ADHD_cost_func_coeffs.npy")
print(coeffs_list.shape)

(40, 4)


In [8]:
# A - Target, B - Predicted
def costfunc(A, B):
    global coeffs
    global X_train
    global psi_cost
    global Laplacian
    global A_transform_cost

    B_tmp = tf.squeeze(B)
    data_fidelity = (tf.norm(tf.multiply(psi_cost, (X_train - B))) ** 2) * coeffs[0]
    smoothness = tf.linalg.trace(tf.transpose(B_tmp) @ Laplacian @ B_tmp @ A_transform_cost) * coeffs[1]
    L_cost = (tf.norm(Laplacian) ** 2) * coeffs[2]
    Z_cost = (tf.norm(A_transform_cost) ** 2) * coeffs[3]
    
    tmp = (smoothness) + (data_fidelity) + (L_cost) + (Z_cost)
    
    global loss_model
    loss_model.append(tmp)
    
    return tmp
    
def costfunc1(A, B):
    global X_train
    global psi_cost
    
    data_fidelity = (tf.norm(tf.multiply(psi_cost, (X_train - B))) ** 2)
    return (data_fidelity)

def costfunc2(A, B):
    global Laplacian
    global A_transform_cost
    
    B_tmp = tf.squeeze(B)
    smoothness = tf.linalg.trace(tf.transpose(B_tmp) @ Laplacian @ B_tmp @ A_transform_cost)
    
    return (smoothness)

def costfunc3(A, B):
    global Laplacian
    
    L_cost = (tf.norm(Laplacian) ** 2)
    
    return (L_cost)

def costfunc4(A, B):
    global A_transform_cost
    
    Z_cost = (tf.norm(A_transform_cost) ** 2)
    
    return (Z_cost)

In [9]:
def mse_unknown(A, B):
    
    global psi_cost
    psi = psi_cost
    
    psid = tf.convert_to_tensor(np.ones([N, T], dtype = float) - psi, dtype = "float32")
    mse_unk = (LA.norm(np.multiply(psid, (X_train - B))) ** 2) / np.sum(psid.numpy().flatten())
    
    global mse_unknown_list
    mse_unknown_list.append(mse_unk)
    
    return mse_unk

def mse_known(A, B):
    
    global psi_cost
    psi = psi_cost
    
    mse_kn = (LA.norm(np.multiply(psi, (X_train - B))) ** 2) / np.sum(psi.numpy().flatten())
    
    global model_pred
    model_pred.append([A.numpy(), B.numpy()])
    
    global mse_known_list
    mse_known_list.append(mse_kn)
    
    return mse_kn

In [10]:
callb = EarlyStopping(monitor = 'loss', mode = 'min', min_delta = 0.0001, verbose = 1, patience = 10)

lr_decay_factor = 1.2
def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr / lr_decay_factor

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [11]:
class ProposedModel(tf.keras.Model):
    
    def __init__(self, L_init, reg_init, loops_init, d_init, p_init, alpha_init, beta_init, threshold_in, **kwargs):
        super(ProposedModel, self).__init__(**kwargs)
        
        self.p1_init = tf.constant(p_init[0], shape = (1, 1), dtype = "float32")
        self.p1 = tf.Variable(initial_value = self.p1_init, trainable = True, name = "p1")
        
        self.p2_init = tf.constant(p_init[1], shape = (1, 1), dtype = "float32")
        self.p2 = tf.Variable(initial_value = self.p2_init, trainable = True, name = "p2")
        
        self.L_in = L_init
        self.preg = reg_init
        self.loops = loops_init
        self.d_int = d_init
        self.alp = alpha_init
        self.bet = beta_init
        self.threshold_init = threshold_in
        
    def data_inpainting(self, psi, Y, Lk, AAT):
        loop_b, reg = self.loops[1], self.preg

        Xk = tf.zeros_like(Y)
        Zk = -(tf.multiply(psi, Xk) - Y + (2 * reg * (Lk @ Xk @ AAT)))
        
        for i in range(loop_b):
            fdx_xk = tf.multiply(psi, Xk) - Y + (2 * reg * (Lk @ Xk @ AAT))

            # row_print = 4
            # tmp = list(zip((tf.multiply(psi, Xk) - Y)[:, row_print].numpy(), (2 * reg * (Lk @ Xk @ AAT)[:, row_print].numpy())))
            # print("\n\nStart here")
            # for k in tmp:
            #     print(k)
            
            fdx_zk = tf.multiply(psi, Zk) - Y + (2 * reg * (Lk @ Zk @ AAT))
            
            tau = tf.linalg.trace(tf.transpose(fdx_xk) @ Zk) / tf.linalg.trace(tf.transpose((Y + fdx_zk)) @ Zk)
            
            Xk_1 = Xk - (tau * Zk)

            fdx_xk_1 = tf.multiply(psi, Xk_1) - Y + (2 * reg * (Lk @ Xk_1 @ AAT))
            gamma = (tf.norm(fdx_xk_1) ** 2) / (tf.norm(fdx_xk) ** 2)

            Zk_1 = (gamma * Zk) - fdx_xk_1

            Xk = Xk_1
            Zk = Zk_1
        # print("\n\nEnds here\n\n")
        return Xk_1 
    
    def graph_learning(self, X, Lk, AAT):
        loop_c, alpha, beta = self.loops[2], self.alp, self.bet
        
        # Computing the smoothness term
        X_AAT_XT = tf.matmul(X, tf.matmul(AAT, tf.transpose(X)))
        
        for i in range(loop_c):

            # Extracting the adjacency matrix from the Laplacian
            adj_i = tf.multiply(Mul, Lk)

            # Computing the gradient with respect to laplacian
            fdL_Lk = X_AAT_XT + (beta * Lk)
            
            # row_print = 4
            # print("For beta")
            # tmp = list(zip(X_AAT_XT[row_print, :].numpy(), np.array(beta * Lk)[row_print, :]))
            # for k in tmp:
            #     print(k)
            
            # Extracting the adjacency matrix from the gradient
            adj_grad = tf.multiply(Mul, fdL_Lk)
            
            # Computing the new adjacency
            # tmp = list(zip(adj_i[row_print, :].numpy(), (alpha * adj_grad)[row_print, :].numpy()))
            # if i == (loop_c-1):
            #     print("\nFor alpha")
            #     for k in tmp:
            #         print(k)
            #     print("\nStart here")
            
            adj_i1 = adj_i - (alpha * adj_grad)
            
            # Avoiding any negative values
            adj_i1 = tf.nn.relu(adj_i1)

            # Computing the new laplacian
            Lk_1 = tf.linalg.diag(tf.math.reduce_sum(adj_i1, 1)) - adj_i1
            
            # Updating the laplacian
            Lk = Lk_1
        # print("\n\nEnds here\n\n")
        return Lk
    

    def call(self, y):

        psi = y[0,:,T:]
        global psi_cost
        psi_cost = psi

        psi = tf.convert_to_tensor(psi, dtype = "float32")
        
        X_in = y[0,:,:T]
        X_in = psi * tf.convert_to_tensor(X_in, dtype = "float32")
        Y = X_in
        
        d = self.d_int
        LT = tf.matmul(d, tf.transpose(d))
        LT = tf.convert_to_tensor(LT, dtype = "float32")
        
        A_transform = tf.eye(T) + (self.p1 * MP(LT, 1)) + (self.p2 * MP(LT, 2))
        AAT = A_transform
        
        global A_transform_cost
        A_transform_cost = AAT
        
        loop_a = self.loops[0]
        threshold = self.threshold_init
        Lk = self.L_in
        
        for i in range(loop_a):
            
            Xk_1 = self.data_inpainting(psi, Y, Lk, AAT)
            
            Xk_1_gt = X_in + tf.multiply((tf.ones(psi.shape) - psi), Xk_1)
            
            Lk_1 = self.graph_learning(Xk_1_gt, Lk, AAT)
            
            Adj = tf.linalg.diag(tf.linalg.diag_part(Lk_1)) - Lk_1 # Gives adjacency
            
            n = tf.math.count_nonzero(Adj, dtype="float32")
            mean = tf.math.reduce_sum(Adj) / n
            Adj = tf.math.subtract(Adj, (threshold * mean)) # Ensures the sparsity of learned graph
            
            Adj = tf.nn.relu(Adj)
            # Adj = tf.math.sign(Adj)
            
            Lk_1 = tf.linalg.diag(tf.math.reduce_sum(Adj, 1)) - Adj
            
            Lk = Lk_1
        
            global Laplacian
            Laplacian = Lk
        
        Xk_1 = self.data_inpainting(psi, Y, Lk, AAT)
        
        global learned_graph
        learned_graph.append(Lk.numpy())
        
        return tf.reshape(Xk_1, [1, N, T])

In [12]:
loop_a = 5
loop_b = 10 # Data Inpainting
loop_c = 10 # Graph Learning

reg_init = 1.0e-2

alpha_init = 1.0e-5
beta_init = 1.0e-2

threshold_init = 0.0

no_of_psi = 20
sensing_ratio = 0.1

p_init = [30.0, 60.0]
lr_decay_factor = 1.2

In [13]:
all_vars = []
for dat in range(num_samples):
    print(f"Datapoint: {dat + 1}")
    T = T_list[dat]
    d = generate_delta(T)
    model_pred = []
    loss_model = []
    learned_graph = []

    X_train = X_train_data[dat]
    Lap_init = initial_graph(X_train, N, 6)
    
    all_psi = []
        
    M = int(sensing_ratio * T)
    X_train_missing = np.zeros([no_of_psi, N, T])
    X_train_concatenated = np.zeros([no_of_psi, N, 2 * T])

    for i in range(no_of_psi):
        psi_k = np.array([0] * (N * M) + [1] * (N * (T - M)))
        np.random.shuffle(psi_k)
        psi_k = psi_k.reshape([N, T])
        all_psi.append(psi_k)
        X_train_missing[i, : , : ] = X_train * psi_k
        X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
    
    coeffs = coeffs_list[dat, :]

    sgd_optimizer = tf.keras.optimizers.SGD(learning_rate = 0.05, momentum = 0.0)
    
    our_model = ProposedModel(Lap_init, reg_init, [loop_a, loop_b, loop_c], 
                              d, p_init, alpha_init, beta_init, threshold_init, name = 'our')
    our_model.compile(optimizer = sgd_optimizer, loss = costfunc,
                      metrics = [mse_known, mse_unknown], run_eagerly=True)
    
    our_model.fit(X_train_concatenated, X_train_missing, epochs = 30, 
                  callbacks = [lr_scheduler], batch_size = 1)
    
    var = our_model.variables
    var_lst = []
    for v in var:
        var_lst.append(v.numpy()[0, 0])
    print(f"\nFilter Coefficients:", var_lst)
    all_vars.append(var_lst)

    graph = learned_graph[-1]
    print(f"\nLearned Graph Shape:\n{graph.shape}")

    np.save(f"./Outputs/Graphs/ADHD/{label_print(dat, 3)}_{y_train_data[dat]}.npy", graph)

Datapoint: 1
Epoch 1/30
20/20 [==============================] - 15s 734ms/step - loss: 90.4204 - mse_known: 0.7250 - mse_unknown: 0.8410 - lr: 0.0500
Epoch 2/30
20/20 [==============================] - 16s 780ms/step - loss: 85.6510 - mse_known: 0.7147 - mse_unknown: 0.8351 - lr: 0.0500
Epoch 3/30
20/20 [==============================] - 15s 769ms/step - loss: 81.1765 - mse_known: 0.7044 - mse_unknown: 0.8290 - lr: 0.0500
Epoch 4/30
20/20 [==============================] - 15s 773ms/step - loss: 77.2973 - mse_known: 0.6947 - mse_unknown: 0.8235 - lr: 0.0417
Epoch 5/30
20/20 [==============================] - 15s 749ms/step - loss: 74.2344 - mse_known: 0.6865 - mse_unknown: 0.8188 - lr: 0.0347
Epoch 6/30
20/20 [==============================] - 15s 773ms/step - loss: 71.7935 - mse_known: 0.6798 - mse_unknown: 0.8149 - lr: 0.0289
Epoch 7/30
20/20 [==============================] - 15s 759ms/step - loss: 69.8337 - mse_known: 0.6740 - mse_unknown: 0.8116 - lr: 0.0241
Epoch 8/30
20/20 [===


Filter Coefficients: [24.15418, 45.265095]

Learned Graph Shape:
(39, 39)
Datapoint: 2
Epoch 1/30
20/20 [==============================] - 15s 768ms/step - loss: 122.5904 - mse_known: 0.6944 - mse_unknown: 0.8166 - lr: 0.0500
Epoch 2/30
20/20 [==============================] - 15s 774ms/step - loss: 117.2908 - mse_known: 0.6826 - mse_unknown: 0.8094 - lr: 0.0500
Epoch 3/30
20/20 [==============================] - 15s 768ms/step - loss: 112.2401 - mse_known: 0.6706 - mse_unknown: 0.8019 - lr: 0.0500
Epoch 4/30
20/20 [==============================] - 15s 753ms/step - loss: 107.7944 - mse_known: 0.6592 - mse_unknown: 0.7948 - lr: 0.0417
Epoch 5/30
20/20 [==============================] - 15s 761ms/step - loss: 104.2347 - mse_known: 0.6496 - mse_unknown: 0.7889 - lr: 0.0347
Epoch 6/30
20/20 [==============================] - 16s 780ms/step - loss: 101.3648 - mse_known: 0.6416 - mse_unknown: 0.7838 - lr: 0.0289
Epoch 7/30
20/20 [==============================] - 15s 773ms/step - loss: 99.


Filter Coefficients: [22.669067, 44.54696]

Learned Graph Shape:
(39, 39)
Datapoint: 3
Epoch 1/30
20/20 [==============================] - 15s 762ms/step - loss: 128.3815 - mse_known: 0.6904 - mse_unknown: 0.7946 - lr: 0.0500
Epoch 2/30
20/20 [==============================] - 16s 786ms/step - loss: 122.3457 - mse_known: 0.6769 - mse_unknown: 0.7863 - lr: 0.0500
Epoch 3/30
20/20 [==============================] - 15s 750ms/step - loss: 116.6006 - mse_known: 0.6631 - mse_unknown: 0.7775 - lr: 0.0500
Epoch 4/30
20/20 [==============================] - 15s 739ms/step - loss: 111.5480 - mse_known: 0.6500 - mse_unknown: 0.7694 - lr: 0.0417
Epoch 5/30
20/20 [==============================] - 15s 736ms/step - loss: 107.5048 - mse_known: 0.6388 - mse_unknown: 0.7625 - lr: 0.0347
Epoch 6/30
20/20 [==============================] - 14s 727ms/step - loss: 104.2455 - mse_known: 0.6294 - mse_unknown: 0.7565 - lr: 0.0289
Epoch 7/30
20/20 [==============================] - 15s 754ms/step - loss: 101


Filter Coefficients: [21.831476, 43.694084]

Learned Graph Shape:
(39, 39)
Datapoint: 4
Epoch 1/30
20/20 [==============================] - 15s 741ms/step - loss: 124.0817 - mse_known: 0.7166 - mse_unknown: 0.8020 - lr: 0.0500
Epoch 2/30
20/20 [==============================] - 15s 770ms/step - loss: 118.8082 - mse_known: 0.7051 - mse_unknown: 0.7953 - lr: 0.0500
Epoch 3/30
20/20 [==============================] - 15s 770ms/step - loss: 113.7818 - mse_known: 0.6934 - mse_unknown: 0.7883 - lr: 0.0500
Epoch 4/30
20/20 [==============================] - 15s 772ms/step - loss: 109.3560 - mse_known: 0.6823 - mse_unknown: 0.7817 - lr: 0.0417
Epoch 5/30
20/20 [==============================] - 15s 768ms/step - loss: 105.8106 - mse_known: 0.6729 - mse_unknown: 0.7760 - lr: 0.0347
Epoch 6/30
20/20 [==============================] - 15s 770ms/step - loss: 102.9506 - mse_known: 0.6649 - mse_unknown: 0.7712 - lr: 0.0289
Epoch 7/30
20/20 [==============================] - 15s 771ms/step - loss: 10


Filter Coefficients: [22.844782, 44.497047]

Learned Graph Shape:
(39, 39)
Datapoint: 5
Epoch 1/30
20/20 [==============================] - 13s 639ms/step - loss: 264.1730 - mse_known: 0.7283 - mse_unknown: 0.8989 - lr: 0.0500
Epoch 2/30
20/20 [==============================] - 13s 636ms/step - loss: 212.0626 - mse_known: 0.6932 - mse_unknown: 0.8775 - lr: 0.0500
Epoch 3/30
20/20 [==============================] - 13s 638ms/step - loss: 170.7455 - mse_known: 0.6573 - mse_unknown: 0.8551 - lr: 0.0500
Epoch 4/30
20/20 [==============================] - 12s 624ms/step - loss: 140.2047 - mse_known: 0.6240 - mse_unknown: 0.8344 - lr: 0.0417
Epoch 5/30
20/20 [==============================] - 13s 649ms/step - loss: 119.2965 - mse_known: 0.5967 - mse_unknown: 0.8165 - lr: 0.0347
Epoch 6/30
20/20 [==============================] - 13s 642ms/step - loss: 104.4969 - mse_known: 0.5743 - mse_unknown: 0.8020 - lr: 0.0289
Epoch 7/30
20/20 [==============================] - 12s 607ms/step - loss: 93

...


Filter Coefficients: [22.50545, 40.607704]

Learned Graph Shape:
(39, 39)
Datapoint: 36
Epoch 1/30
20/20 [==============================] - 15s 768ms/step - loss: 117.3834 - mse_known: 0.7725 - mse_unknown: 0.9101 - lr: 0.0500
Epoch 2/30
20/20 [==============================] - 15s 735ms/step - loss: 112.8880 - mse_known: 0.7634 - mse_unknown: 0.9051 - lr: 0.0500
Epoch 3/30
20/20 [==============================] - 15s 747ms/step - loss: 108.5928 - mse_known: 0.7539 - mse_unknown: 0.9000 - lr: 0.0500
Epoch 4/30
20/20 [==============================] - 15s 751ms/step - loss: 104.8024 - mse_known: 0.7450 - mse_unknown: 0.8952 - lr: 0.0417
Epoch 5/30
20/20 [==============================] - 15s 729ms/step - loss: 101.7601 - mse_known: 0.7373 - mse_unknown: 0.8910 - lr: 0.0347
Epoch 6/30
20/20 [==============================] - 15s 747ms/step - loss: 99.3021 - mse_known: 0.7308 - mse_unknown: 0.8874 - lr: 0.0289
Epoch 7/30
20/20 [==============================] - 15s 749ms/step - loss: 97.


Filter Coefficients: [23.39733, 45.624557]

Learned Graph Shape:
(39, 39)
Datapoint: 37
Epoch 1/30
20/20 [==============================] - 18s 891ms/step - loss: 117.8819 - mse_known: 0.6694 - mse_unknown: 0.7763 - lr: 0.0500
Epoch 2/30
20/20 [==============================] - 18s 906ms/step - loss: 109.5571 - mse_known: 0.6545 - mse_unknown: 0.7665 - lr: 0.0500
Epoch 3/30
20/20 [==============================] - 18s 899ms/step - loss: 101.9221 - mse_known: 0.6395 - mse_unknown: 0.7566 - lr: 0.0500
Epoch 4/30
20/20 [==============================] - 18s 882ms/step - loss: 95.4433 - mse_known: 0.6256 - mse_unknown: 0.7475 - lr: 0.0417
Epoch 5/30
20/20 [==============================] - 18s 886ms/step - loss: 90.4239 - mse_known: 0.6141 - mse_unknown: 0.7398 - lr: 0.0347
Epoch 6/30
20/20 [==============================] - 17s 870ms/step - loss: 86.4863 - mse_known: 0.6045 - mse_unknown: 0.7335 - lr: 0.0289
Epoch 7/30
20/20 [==============================] - 18s 872ms/step - loss: 83.36


Filter Coefficients: [23.00281, 40.946827]

Learned Graph Shape:
(39, 39)
Datapoint: 38
Epoch 1/30
20/20 [==============================] - 16s 781ms/step - loss: 125.4486 - mse_known: 0.7152 - mse_unknown: 0.8233 - lr: 0.0500
Epoch 2/30
20/20 [==============================] - 15s 771ms/step - loss: 119.9406 - mse_known: 0.7033 - mse_unknown: 0.8161 - lr: 0.0500
Epoch 3/30
20/20 [==============================] - 15s 773ms/step - loss: 114.6960 - mse_known: 0.6911 - mse_unknown: 0.8086 - lr: 0.0500
Epoch 4/30
20/20 [==============================] - 15s 752ms/step - loss: 110.0813 - mse_known: 0.6797 - mse_unknown: 0.8015 - lr: 0.0417
Epoch 5/30
20/20 [==============================] - 15s 772ms/step - loss: 106.3871 - mse_known: 0.6699 - mse_unknown: 0.7955 - lr: 0.0347
Epoch 6/30
20/20 [==============================] - 15s 745ms/step - loss: 103.4093 - mse_known: 0.6616 - mse_unknown: 0.7902 - lr: 0.0289
Epoch 7/30
20/20 [==============================] - 15s 758ms/step - loss: 10


Filter Coefficients: [22.573952, 44.23363]

Learned Graph Shape:
(39, 39)
Datapoint: 39
Epoch 1/30
20/20 [==============================] - 18s 877ms/step - loss: 111.7628 - mse_known: 0.6886 - mse_unknown: 0.7945 - lr: 0.0500
Epoch 2/30
20/20 [==============================] - 17s 868ms/step - loss: 104.3456 - mse_known: 0.6743 - mse_unknown: 0.7854 - lr: 0.0500
Epoch 3/30
20/20 [==============================] - 17s 827ms/step - loss: 97.5045 - mse_known: 0.6598 - mse_unknown: 0.7761 - lr: 0.0500
Epoch 4/30
20/20 [==============================] - 17s 870ms/step - loss: 91.6678 - mse_known: 0.6462 - mse_unknown: 0.7673 - lr: 0.0417
Epoch 5/30
20/20 [==============================] - 17s 867ms/step - loss: 87.1242 - mse_known: 0.6349 - mse_unknown: 0.7599 - lr: 0.0347
Epoch 6/30
20/20 [==============================] - 17s 872ms/step - loss: 83.5457 - mse_known: 0.6255 - mse_unknown: 0.7537 - lr: 0.0289
Epoch 7/30
20/20 [==============================] - 18s 878ms/step - loss: 80.700


Filter Coefficients: [22.898584, 42.04416]

Learned Graph Shape:
(39, 39)
Datapoint: 40
Epoch 1/30
20/20 [==============================] - 11s 558ms/step - loss: 122.7029 - mse_known: 0.6715 - mse_unknown: 0.7924 - lr: 0.0500
Epoch 2/30
20/20 [==============================] - 11s 560ms/step - loss: 117.2341 - mse_known: 0.6588 - mse_unknown: 0.7842 - lr: 0.0500
Epoch 3/30
20/20 [==============================] - 11s 559ms/step - loss: 112.0182 - mse_known: 0.6457 - mse_unknown: 0.7757 - lr: 0.0500
Epoch 4/30
20/20 [==============================] - 11s 557ms/step - loss: 107.4226 - mse_known: 0.6335 - mse_unknown: 0.7676 - lr: 0.0417
Epoch 5/30
20/20 [==============================] - 11s 551ms/step - loss: 103.7392 - mse_known: 0.6230 - mse_unknown: 0.7609 - lr: 0.0347
Epoch 6/30
20/20 [==============================] - 11s 541ms/step - loss: 100.7665 - mse_known: 0.6142 - mse_unknown: 0.7551 - lr: 0.0289
Epoch 7/30
20/20 [==============================] - 11s 542ms/step - loss: 98

In [14]:
np.save("./Outputs/ADHD_mse_known.npy", mse_known_list)
np.save("./Outputs/ADHD_mse_unknown.npy", mse_unknown_list)